In [1]:
import panel as pn
import param as pm
import pandas as pd
from glob import glob
from bokeh.palettes import Category20, Turbo256
pn.extension()

# Assuming other imports and setup are done as before

class Simulation(pm.Parameterized):
    experiment = pm.ObjectSelector(default='sanity_check_run', objects=sorted(list(set([f.split('/')[-1].split('-')[0] for f in glob('../data/simulations/*')]))))
    dataset = pm.Selector(default=None)  # This will be dynamically populated
    drop_list = pm.List(precedence=-1, default=['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'])
    sim_df = pm.DataFrame(precedence=-1)
    max_rows = pm.Integer(6, bounds=(1, None), step=2)
    color_palette = pm.Selector(default=Category20, objects=[Category20, Turbo256], precedence=-1)
    column_colors = pm.Dict(precedence=-1)

    def __init__(self, **params):
        super(Simulation, self).__init__(**params)
        self._update_dataset_options()
        self._load_simulation_data()

    @pm.depends('experiment', watch=True)
    def _update_dataset_options(self, event=None):
        datasets = sorted(glob(f"../data/simulations/{self.experiment}*"))
        self.param.dataset.objects = datasets
        self.dataset = datasets[-1] if datasets else None

    @pm.depends('dataset', 'drop_list', watch=True)
    def _load_simulation_data(self):
        """Load the simulation data when dataset or drop_list are changed."""
        if self.dataset:
            self.sim_df = pd.read_pickle(self.dataset).drop(self.drop_list, axis=1)
        else:
            self.sim_df = pd.DataFrame()

    @pm.depends('color_palette', watch=True)
    def _set_column_colors(self):
        columns = self.sim_df.column
        if self.color_palette == Turbo256:
            column_colors = dict(zip(self.sim_df.columns, [self.color_palette[int(i)] for i in np.linspace(0,len(self.color_palette)-1, len(self.sim_df.columns))]))
        
        if self.color_palette == Category20:
            column_colors = {col: self.color_palette[20][i%20] for i, col in enumerate(self.sim_df.columns)}
    

    def view_sim_df_results(self):
        return pn.panel(self.sim_df, max_rows=self.max_rows)

    def runs_overview(self):
        return self.sim_df.groupby(['run', 'label', 'environmental_label']).size().reset_index(name='Days')

    def view_runs_overview(self):
        return pn.panel(self.runs_overview(), max_rows=self.max_rows)

    def view(self):
        """View the selected simulation results."""
        pd.set_option('display.max_rows', self.max_rows)
        view = pn.Column(
            """
            ## Simulation Analysis Template
            """,
            pn.Accordion(
                ('Select Parameters', self), 
                ('Results DataFrame', self.view_sim_df_results), 
                ('Runs Overview', self.view_runs_overview),
            ),
        )
        view[1].active = list(range(len(view[1])))
        return view


# Usage
s = Simulation()
s.view()


Column
    [0] Markdown(str)
    [1] Accordion(active=[0, 1, 2])
        [0] Column(margin=(5, 10), name='Select Parameters')
            [0] StaticText(value='<b>Select Parameters</b>'...)
            [1] Select(name='Experiment', options=OrderedDict([('fund_inclus...]), value='sanity_check_run')
            [2] Select(name='Dataset', options=OrderedDict([('../data/sim...]), value='../data/simulations/sanit...)
            [3] IntInput(name='Max rows', start=1, step=2, value=6)
        [1] ParamMethod(method, _pane=DataFrame, defer_load=False, name='Results DataFrame')
        [2] ParamMethod(method, _pane=DataFrame, defer_load=False, name='Runs Overview')

In [2]:
s.sim_df.columns

Index(['days_passed', 'blocks_passed', 'circulating_supply', 'user_supply',
       'earned_supply', 'issued_supply', 'earned_minus_burned_supply',
       'total_supply', 'sum_of_stocks', 'storage_fee_per_rewards',
       'block_utilization', 'dsf_relative_disbursal_per_day',
       'reward_issuance_balance', 'other_issuance_balance',
       'operators_balance', 'nominators_balance', 'holders_balance',
       'farmers_balance', 'staking_pool_balance', 'fund_balance',
       'burnt_balance', 'nominator_pool_shares', 'operator_pool_shares',
       'block_reward', 'blockchain_history_size', 'total_space_pledged',
       'allocated_tokens', 'buffer_size', 'reference_subsidy',
       'average_priority_fee', 'average_compute_weight_per_tx',
       'average_transaction_size', 'transaction_count',
       'average_compute_weight_per_bundle', 'average_bundle_size',
       'bundle_count', 'compute_fee_multiplier', 'compute_fee_volume',
       'free_space', 'transaction_byte_fee', 'extrinsic_length

In [ ]:
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}

In [ ]:
header_styles = [{
    'selector': f'th.col_heading.level0.col{i}',
    'props': [('background-color', column_colors.get(col))]
} for i, col in enumerate(df.columns)]

df.columns.name = 'balance'

describe_difference_df_styled = df.style.map(color_scale).set_table_styles(header_styles)
describe_difference_df_styled